# FIT File Support

> Load and process FIT files from Garmin devices

In [ ]:
#| default_exp fit

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import fitdecode
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Optional, Tuple, List
import warnings

## FIT File Loader

The `FitLoader` class provides functionality to load and extract power data from Garmin FIT files.

In [ ]:
#| export
class FitLoader:
    """Load and extract data from Garmin FIT files"""
    
    def __init__(self, filepath: str):
        """Initialize with path to FIT file"""
        self.filepath = Path(filepath)
        if not self.filepath.exists():
            raise FileNotFoundError(f"FIT file not found: {filepath}")
        if not self.filepath.suffix.lower() == '.fit':
            warnings.warn(f"File extension is not .fit: {filepath}")
    
    def extract_power_data(self) -> pd.DataFrame:
        """Extract power and time data from FIT file
        
        Returns:
            DataFrame with columns: timestamp, power, elapsed_time
        """
        records = []
        start_time = None
        
        with fitdecode.FitReader(self.filepath) as fit:
            for frame in fit:
                if isinstance(frame, fitdecode.FitDataMessage):
                    if frame.name == 'record':
                        record = {}
                        
                        # Extract relevant fields
                        for field in frame.fields:
                            if field.name == 'timestamp':
                                record['timestamp'] = field.value
                                if start_time is None:
                                    start_time = field.value
                            elif field.name == 'power':
                                record['power'] = field.value
                        
                        # Only keep records with power data
                        if 'power' in record and record['power'] is not None:
                            if 'timestamp' in record:
                                # Calculate elapsed time in seconds
                                elapsed = (record['timestamp'] - start_time).total_seconds()
                                record['elapsed_time'] = elapsed
                            records.append(record)
        
        if not records:
            raise ValueError("No power data found in FIT file")
        
        df = pd.DataFrame(records)
        
        # Ensure we have elapsed_time column
        if 'elapsed_time' not in df.columns and 'timestamp' in df.columns:
            df['elapsed_time'] = (df['timestamp'] - df['timestamp'].iloc[0]).dt.total_seconds()
        
        return df
    
    def get_power_duration_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """Extract power and duration data suitable for MMP/PDC analysis
        
        Returns:
            Tuple of (durations, powers) as numpy arrays
        """
        df = self.extract_power_data()
        
        # Use elapsed time as duration and power values
        durations = df['elapsed_time'].values
        powers = df['power'].values
        
        # Remove any NaN or invalid values
        valid_mask = ~(np.isnan(powers) | np.isnan(durations))
        durations = durations[valid_mask]
        powers = powers[valid_mask]
        
        return durations, powers
    
    def compute_mmp_curve(self, durations: Optional[List[int]] = None) -> Tuple[np.ndarray, np.ndarray]:
        """Compute Mean Maximal Power curve from FIT file data
        
        Args:
            durations: List of durations in seconds to compute MMP for.
                      If None, uses default durations from 1s to 3600s
        
        Returns:
            Tuple of (durations, mmp_values) as numpy arrays
        """
        if durations is None:
            # Default durations: 1s to 1 hour with more resolution at shorter durations
            durations = list(range(1, 61)) + list(range(60, 301, 5)) + list(range(300, 1801, 30)) + list(range(1800, 3601, 60))
        
        df = self.extract_power_data()
        powers = df['power'].values
        
        mmp_values = []
        
        for duration in durations:
            if duration > len(powers):
                # Not enough data for this duration
                mmp_values.append(np.nan)
                continue
            
            # Calculate rolling mean for this duration
            rolling_mean = pd.Series(powers).rolling(window=duration, min_periods=duration).mean()
            max_power = rolling_mean.max()
            
            mmp_values.append(max_power if not np.isnan(max_power) else 0)
        
        # Remove NaN values
        durations = np.array(durations)
        mmp_values = np.array(mmp_values)
        valid_mask = ~np.isnan(mmp_values)
        
        return durations[valid_mask], mmp_values[valid_mask]

## Integration with MMP and PDC Classes

Add convenience methods to create MMP and PDC objects from FIT files.

In [ ]:
#| export
def load_fit_file(filepath: str) -> FitLoader:
    """Load a FIT file and return a FitLoader instance
    
    Args:
        filepath: Path to the FIT file
    
    Returns:
        FitLoader instance
    """
    return FitLoader(filepath)

In [ ]:
#| export
def mmp_from_fit(filepath: str, durations: Optional[List[int]] = None):
    """Create an MMP object from a FIT file
    
    Args:
        filepath: Path to the FIT file
        durations: List of durations in seconds to compute MMP for
    
    Returns:
        MMP object with data from the FIT file
    """
    from .mmp import MMP
    
    loader = FitLoader(filepath)
    x, y = loader.compute_mmp_curve(durations)
    
    return MMP(x, y)

In [ ]:
#| export
def pdc_from_fit(filepath: str, durations: Optional[List[int]] = None):
    """Create a PDC object from a FIT file
    
    Args:
        filepath: Path to the FIT file
        durations: List of durations in seconds to compute MMP for
    
    Returns:
        PDC object with data from the FIT file
    """
    from .pdc import PDC
    
    loader = FitLoader(filepath)
    x, y = loader.compute_mmp_curve(durations)
    
    return PDC(x, y)

## Example Usage

Here's how to use the FIT file functionality:

In [ ]:
#| eval: false
# Load a FIT file
fit_loader = load_fit_file('path/to/your/activity.fit')

# Extract raw power data
power_df = fit_loader.extract_power_data()
print(power_df.head())

# Compute MMP curve
durations, mmp_powers = fit_loader.compute_mmp_curve()

# Create MMP object directly from FIT file
mmp = mmp_from_fit('path/to/your/activity.fit')

# Create PDC object directly from FIT file
pdc = pdc_from_fit('path/to/your/activity.fit')

# Fit the power duration curve
result = pdc.fit()
print(result.best_values)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()